# Add new columns to a Datafeed

To use Backtrader, the standard process is to import datafeeds with basic information like prices (OHLC), volumes and open interests. However, I can be useful to add other fields such as:  
- **Indicators**: This way, it is faster to develop and run strategies in Backtrader as the results of the indicators are already available and we do not need to create complex Strategy functions to be computed inside loops.  
- **Fundamental or sentiment data**: anything can be added as extra columns.

Each column added to a csv file will be considered a a 'line' by Backtrader. This is an extension of standard lines which are open, high, low, close, volume, openinterest.

To do this, create a class inheriting from **backtrader.feeds.GenericCSVData**.  
See https://www.backtrader.com/docu/extending-a-datafeed/#plotting-that-extra-pe-line

In [22]:
# import libraries
import pandas as pd
import backtrader as bt
import datetime  # For datetime objects

## 1. Create csv file with a new 'SMA' column

First step is to create a csv file with a new column. For exemple, let's calculate a 20-day Simple Moving Average (SMA) using Pandas.

In [23]:
# Load csv file into a DataFrame
dataframe = pd.read_csv('data/orcl-1995-2014.csv',
           index_col=0)
dataframe.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-12-24,46.360001,46.709999,46.150002,46.230000,43.488419,10238200
2014-12-26,46.189999,46.500000,46.070000,46.099998,43.366119,6901500
2014-12-29,46.020000,46.090000,45.599998,45.610001,42.905186,9701400
2014-12-30,45.549999,45.660000,45.290001,45.340000,42.651192,9968400
2014-12-31,45.450001,45.560001,44.970001,44.970001,42.303135,13269200


Now let's add a column 'SMA' to calculate a 20-day MA

In [25]:
# Add new column SMA
dataframe['SMA']=dataframe['Close'].rolling(20).mean()
dataframe

,Open,High,Low,Close,Adj Close,Volume,SMA
Date,,,,,,,
1995-01-03,2.179012,2.191358,2.117284,2.117284,1.883304,36301200,NaN
1995-01-04,2.123457,2.148148,2.092592,2.135803,1.899776,46051600,NaN
1995-01-05,2.141975,2.148148,2.086420,2.092592,1.861340,37762800,NaN
1995-01-06,2.092592,2.154321,2.061728,2.117284,1.883304,41864400,NaN
1995-01-09,2.135803,2.179012,2.129630,2.179012,1.938211,34639200,NaN
...,...,...,...,...,...,...,...
2014-12-24,46.360001,46.709999,46.150002,46.230000,43.488419,10238200,42.5715
2014-12-26,46.189999,46.500000,46.070000,46.099998,43.366119,6901500,42.7830
2014-12-29,46.020000,46.090000,45.599998,45.610001,42.905186,9701400,42.9430


In [29]:
# Drop lines with NaNs as SMA is not yet available
dataframe.dropna(inplace=True)
dataframe.head()

,Open,High,Low,Close,Adj Close,Volume,SMA
Date,,,,,,,
1995-01-30,2.111111,2.117284,2.055556,2.061728,1.833888,44440000,2.127469
1995-01-31,2.074074,2.129630,2.061728,2.104938,1.872322,37908400,2.126852
1995-02-01,2.129630,2.141975,2.080247,2.092592,1.861340,24384400,2.124691
1995-02-02,2.092592,2.123457,2.080247,2.117284,1.883304,24165600,2.125926
1995-02-03,2.135803,2.197531,2.123457,2.172839,1.932719,53942800,2.128704


In [31]:
# To avoid confusion, we drop column "Adj Close" which is not standard
dataframe.drop(columns=['Adj Close'], inplace=True)

In [33]:
dataframe.head()

,Open,High,Low,Close,Volume,SMA
Date,,,,,,
1995-01-30,2.111111,2.117284,2.055556,2.061728,44440000,2.127469
1995-01-31,2.074074,2.129630,2.061728,2.104938,37908400,2.126852
1995-02-01,2.129630,2.141975,2.080247,2.092592,24384400,2.124691
1995-02-02,2.092592,2.123457,2.080247,2.117284,24165600,2.125926
1995-02-03,2.135803,2.197531,2.123457,2.172839,53942800,2.128704


In [32]:
# Save new file csv file with SMA field
dataframe.to_csv('data/orcl-1995-2014_SMA.csv')

## 2. Create a new trading strategy using new column 'SMA'

We must first create a class inherited from class GenericCSVData that will add a new line called 'sma' to standard lines already available in GenericCSVData ('datetime', 'time', 'open', 'high', 'low', 'close', 'volume', 'openinterest')

In [34]:
from backtrader.feeds import GenericCSVData

class GenericCSV_SMA(GenericCSVData):
    """
    Add new line 'sma' corresponding to column 6 in csv file
    ----
    Make sure SMA column is number 6 in csv file used!
    Other GenericCSVData standard fields will be recognized
    """

    # Add a 'sma' line to the inherited ones from the base class
    lines = ('sma',)

    # add the parameter to the parameters inherited from the base class
    # openinterest in GenericCSVData has index 7 so sma should be 8
    # However, SMA is column 8 in the csv file used so sma should be 6
    params = (('sma', 6),)


We can create a new strategy that uses the SMA column available in the csv file as an input for a trading strategy. Here the strategy is just to print the level of the Simple Moving Average every day.

In [51]:
class TestStrategy(bt.Strategy):
    """
    Print close price in logs every day
    """
    def __init__(self):
        """ Keep a reference to the close line in the datas[0] dataseries"""
        self.dataopen = self.datas[0].lines.open
        self.dataclose = self.datas[0].lines.close
        # Define column SMA of the CSV file as a new line
        self.my_sma = self.datas[0].lines.sma

    def log(self, txt, dt=None):
        """ Logging function for this strategy"""
        dt = dt or self.datas[0].datetime.date(0)
        # print date followed by any message found in 'txt'
        print('%s, %s' % (dt.isoformat(), txt))

    def next(self):
        """ Simply log the closing price of the series from the reference"""
        # output will be of format "date, Close, a float" (i.e. close price)
        self.log('Open, %.2f,  Close, %.2f,  SMA, %.2f' % (self.dataopen[0], self.dataclose[0], self.my_sma[0]))

In [52]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add the strategy (print logs)
    cerebro.addstrategy(TestStrategy)

    # Create a Data Feed
    data = GenericCSV_SMA(
        dataname='data/orcl-1995-2014_SMA.csv',
        # Default date format is '%Y-%m-%d %H:%M:%S'
        # As hours are not available in csv file, change to
        dtformat="%Y-%m-%d",
        # Do not pass values before this date
        fromdate=datetime.datetime(2000, 1, 1),
        # Do not pass values before this date
        todate=datetime.datetime(2000, 12, 31),
        # Do not pass values after this date
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2000-01-03, Open, 31.16,  Close, 29.53,  SMA, 23.65
2000-01-04, Open, 28.88,  Close, 26.92,  SMA, 24.02
2000-01-05, Open, 25.41,  Close, 25.50,  SMA, 24.30
2000-01-06, Open, 25.04,  Close, 24.00,  SMA, 24.55
2000-01-07, Open, 23.75,  Close, 25.84,  SMA, 24.86
2000-01-10, Open, 27.00,  Close, 28.94,  SMA, 25.25
2000-01-11, Open, 28.16,  Close, 28.09,  SMA, 25.66
2000-01-12, Open, 28.06,  Close, 26.41,  SMA, 26.02
2000-01-13, Open, 27.12,  Close, 26.27,  SMA, 26.20
2000-01-14, Open, 27.25,  Close, 26.70,  SMA, 26.41
2000-01-18, Open, 26.97,  Close, 27.81,  SMA, 26.67
2000-01-19, Open, 28.06,  Close, 28.56,  SMA, 26.90
2000-01-20, Open, 29.50,  Close, 29.62,  SMA, 27.16
2000-01-21, Open, 30.75,  Close, 29.84,  SMA, 27.34
2000-01-24, Open, 30.12,  Close, 27.09,  SMA, 27.36
2000-01-25, Open, 27.53,  Close, 28.22,  SMA, 27.46
2000-01-26, Open, 28.38,  Close, 27.53,  SMA, 27.51
2000-01-27, Open, 27.91,  Close, 25.91,  SMA, 27.43
2000-01-28, Open, 25.75,  Cl

All good! Values printed inside Backtrader are the same than those available in the csv file.